# Investigating the USV data from Paper 2

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot, track_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
#plt.style.use('paper')
#from IPython.display import set_matplotlib_formats
#set_matplotlib_formats('pdf')

from src.symbols import *

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]

from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions, create_model_from_motion_regression
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler
from src.prime_system import PrimeSystem

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.pipeline import Pipeline
from src.parameters import df_parameters
p = df_parameters['symbol']
from src.feature_selection import drop_multicollinearity
from src.bias_variance_tradeoff import train_test_split_run
from sklearn.metrics import r2_score
from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions, create_model_from_motion_regression, create_full_model_from_motion_regression
from src.models.regression import Regression
from myst_nb import glue
import src.symbols as s
from IPython.display import Latex
from src.substitute_dynamic_symbols import run
import seaborn as sns
from src.feature_selection import feature_imporance, BestFeatures, DropCorrelation
from wPCC_pipeline.turning_circle import TurningCircle
from src.models.propeller import fit, predict, predictor
from wPCC_pipeline.monte_carlo import monte_carlo, parameter_variation
from wPCC_pipeline.pipelines.preprocess.nodes import load, filter

In [ ]:
%reload_kedro

In [ ]:
ship="USV"
id = "zigzag_30_inter_xy"
#id = "turnningdata_xy"
data = catalog.load(f"{ ship }.{ id }.raw_data")

if id == "zigzag_30_inter_xy":
    #data.index = data.index/10  #?
    cutoff = 0.5
else:
    #data.index = data.index/2  #?
    cutoff = 0.1

In [ ]:
data.head()

In [ ]:
np.mean(np.diff(data.index))

In [ ]:
data.index[-1]

In [ ]:
data.dtypes

In [ ]:
data['-rudder'] = -data['rudder']
data.plot(y=['heading','rudder'])

In [ ]:
fig,ax=plt.subplots()
data['psi'] = np.unwrap(data['psi'])
data.plot(x='y0',y='x0', kind='line', ax=ax)
ax.axis('equal')

In [ ]:
df = load(data)

In [ ]:
df.plot(y='u')

In [ ]:
df.plot(y='v')

In [ ]:


df_filtered = filter(df, cutoff=cutoff)

df_filtered['psi'] = np.unwrap(df_filtered['psi'])
mask = (df_filtered['psi'] - df_filtered.iloc[0]['psi']).abs() < 1.37*(2*np.pi)
df_filtered = df_filtered.loc[mask]

In [ ]:
track_plot(df_filtered, lpp=3, beam=0.7)

fig,ax=plt.subplots()
df_filtered.plot(y='u',ax=ax)
ax.grid()

fig,ax=plt.subplots()
df_filtered.plot(y='v', ax=ax)
ax.grid()

fig,ax=plt.subplots()
df_filtered['r_deg'] = np.rad2deg(df_filtered['r'])
df_filtered.plot(y='r_deg', ax=ax)
ax.grid()